In [69]:
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *
from datetime import *

In [70]:
key_path = key_path
project = project_id

# Sources
currency_table = 'silver_currency_data'
currency_dataset = 'silver'
sp500_table = 'bronze_sp500_tickers'
sp500_dataset = 'bronze'
ticker_table = 'bronze_ticker_data'
ticker_dataset = 'bronze'

# Tables id
table_conca_currency = f'{project}.{currency_dataset}.{currency_table}'
table_conca_sp500 = f'{project}.{sp500_dataset}.{sp500_table}'
table_conca_ticker = f'{project}.{ticker_dataset}.{ticker_table}'

# Loading tables
table_to_save = 'gold_main_sp500'
dataset_to_save = 'gold'



In [71]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [72]:
df = bigquery.run_query(
    f"""
    SELECT
        ti.*,
        cu.EURUSD,
        cu.GBPUSD,
        cu.USDCNY,
        cu.JPYUSD,
        cu.AUDUSD,
        cu.USDCHF
    FROM {table_conca_ticker} AS ti
    INNER JOIN {table_conca_sp500} AS sp
        ON sp.Symbol = ti.ticker    
    INNER JOIN {table_conca_currency} AS cu
        ON ti.date = cu.Date
    """
)

In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411776 entries, 0 to 411775
Data columns (total 14 columns):
 #   Column    Non-Null Count   Dtype              
---  ------    --------------   -----              
 0   date      411776 non-null  datetime64[us, UTC]
 1   open      411776 non-null  float64            
 2   high      411776 non-null  float64            
 3   low       411776 non-null  float64            
 4   close     411776 non-null  float64            
 5   adjclose  411776 non-null  float64            
 6   volume    411776 non-null  Int64              
 7   ticker    411776 non-null  object             
 8   EURUSD    411776 non-null  float64            
 9   GBPUSD    411776 non-null  float64            
 10  USDCNY    411604 non-null  float64            
 11  JPYUSD    411776 non-null  float64            
 12  AUDUSD    411776 non-null  float64            
 13  USDCHF    411776 non-null  float64            
dtypes: Int64(1), datetime64[us, UTC](1), float64(11), ob

In [74]:
# Asegúrate de que la columna 'date' esté en formato datetime y ordena el DataFrame
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by='date')

# Rellenar los valores nulos en 'USDCNY' con el valor anterior
df['USDCNY'] = df['USDCNY'].fillna(method='ffill')
df['EURUSD'] = df['EURUSD'].fillna(method='ffill')
df['GBPUSD'] = df['GBPUSD'].fillna(method='ffill')
df['JPYUSD'] = df['JPYUSD'].fillna(method='ffill')
df['AUDUSD'] = df['AUDUSD'].fillna(method='ffill')
df['USDCHF'] = df['USDCHF'].fillna(method='ffill')


# Verificar los resultados
print(df.info())


<class 'pandas.core.frame.DataFrame'>
Index: 411776 entries, 0 to 308050
Data columns (total 14 columns):
 #   Column    Non-Null Count   Dtype              
---  ------    --------------   -----              
 0   date      411776 non-null  datetime64[us, UTC]
 1   open      411776 non-null  float64            
 2   high      411776 non-null  float64            
 3   low       411776 non-null  float64            
 4   close     411776 non-null  float64            
 5   adjclose  411776 non-null  float64            
 6   volume    411776 non-null  Int64              
 7   ticker    411776 non-null  object             
 8   EURUSD    411776 non-null  float64            
 9   GBPUSD    411776 non-null  float64            
 10  USDCNY    411776 non-null  float64            
 11  JPYUSD    411776 non-null  float64            
 12  AUDUSD    411776 non-null  float64            
 13  USDCHF    411776 non-null  float64            
dtypes: Int64(1), datetime64[us, UTC](1), float64(11), object(

In [75]:
# Guardamos los datos en bigquery
bigquery.save_dataframe(df, project, dataset_to_save, table_to_save, if_exists='append', schema=None)

100%|██████████| 1/1 [00:00<?, ?it/s]
